In [ ]:
#Import some libraries
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.figsize'] = (16, 6)
import seaborn as sns
import numpy as np
import pandas as pd
import os

sns.set_style("dark"); sns.set_palette("muted"); sns.set_context("paper")

from sklearn.datasets import make_sparse_coded_signal



Let's simulate data from a sparse regression

In [ ]:
y, X, beta = make_sparse_coded_signal(n_samples=1,
                                   n_components=500,
                                   n_features=4000,
                                   n_nonzero_coefs=30,
                                   random_state=0)
# add some noise to y
y = y + 0.1 * np.random.randn(len(y))

plt.step(range(len(beta)), beta,  where='mid', lw=1)
plt.plot(range(len(beta)), beta, '.')
plt.xlabel(r'$true \beta$')

Let's create our horseshoe and elastic linear models
The horseshoe (HS)prior is different than the laplacian for an L1 or normal for L2 (ridge)

The HS prior is a mixture model prior

In [ ]:
#Lets import necessary libraries and define some possible models: horeshoe, ridge, lasso, scalemixture.
import numpyro
import numpyro.distributions as dist
import jax.numpy as jnp
def horeshoe_linear_model(y=None, X=None):
    n_predictors = X.shape[1]
    y_sigma = numpyro.sample('y_sigma',dist.Gamma(0.5,1))
    with numpyro.plate('local_shrinkage', n_predictors):
        Lambda = numpyro.sample('lambda', dist.HalfCauchy(scale=1))
        Tau = numpyro.sample('tau',dist.Gamma(0.5,8*Lambda))
        horseshoe_sigma = Lambda*Tau
        Beta = numpyro.sample('beta', dist.Normal(loc=0, scale=horseshoe_sigma))
    mu = jnp.dot(X, Beta)
    numpyro.sample('obs', dist.Normal(loc=mu, scale=y_sigma), obs=y)


def elastic_linear_model(y=None, X=None):
    n_predictors = X.shape[1]
    y_sigma = numpyro.sample('y_sigma',dist.Gamma(0.5,1)) #Prior on observation noise. Model as RV
    with numpyro.plate('local_shrinkage', n_predictors):
        Lambda1 = numpyro.sample('lambda_1', dist.HalfCauchy(scale=1))
        Lambda2 = numpyro.sample('lambda_2', dist.HalfCauchy(scale=1))
        Tau_rate=(8*Lambda2*y_sigma)/(Lambda1**2)
        Tau = numpyro.sample('tau',dist.Gamma(0.5,Tau_rate))
        Beta_Sigma = (Lambda2/y_sigma)*((Tau+1)/((Tau+1)-1))
        Beta = numpyro.sample('beta', dist.Normal(loc=0, scale=Beta_Sigma))
    mu = jnp.dot(X, Beta)
    numpyro.sample('obs', dist.Normal(loc=mu, scale=y_sigma), obs=y)

In [ ]:
from numpyro.infer import MCMC, NUTS
from jax import random

nuts_kernel = NUTS(elastic_linear_model)
mcmc = MCMC(nuts_kernel, num_warmup=500, num_samples=500)
rng_key = random.PRNGKey(0)
mcmc.run(rng_key, y=y, X=X)

In [ ]:
posterior_samples = mcmc.get_samples()
beta_mu = jnp.mean(posterior_samples['beta'], axis=0)
plt.step(range(len(beta)), beta,  where='mid', lw=1)
plt.plot(range(len(beta)), beta, '.')
plt.plot(range(len(beta)), beta_mu, 'g*')
plt.xlabel(r'$\beta$')

In [ ]:
import arviz as az

data = az.from_numpyro(mcmc)
az.plot_trace(data, compact=True, figsize=(15, 25));